In [9]:
import fileio
import pydata

#import numpy as np
#import pandas as pd

In [2]:
from importlib import reload

In [116]:
reload(fileio)
reload(shapes)

<module 'shapes' from '/home/nelson/Data/kba/shapes.py'>

In [117]:
myfile = fileio.InputData('arecaceae.csv')
myfile.iucnFile('arecaceae_categories.csv')

In [118]:
myar = shapes.KBA('/home/nelson/Dropbox/Humboldt/Postdoc/KBA_by_IUCN/Colombia_KBA', 'SitRecID')

In [119]:
myar.spp_inclusion(myfile)

In [98]:
myar.polys

{14405: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec862c8d0>,
  'new_spp': {}},
 14408: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec6c13cd0>,
  'new_spp': {}},
 14414: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec8885b50>,
  'new_spp': {'Acoelorraphe wrightii': [1, 5],
   'Coccothrinax argentata': [4, 0, 5]}},
 14419: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec862c490>,
  'new_spp': {}},
 14423: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec737bc90>,
  'new_spp': {}},
 14429: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec6c13150>,
  'new_spp': {}},
 14433: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec8885cd0>,
  'new_spp': {}},
 14440: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec862cd10>,
  'new_spp': {}},
 14454: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec737b090>,
  'new_spp': {}},
 14456: {'shape': <shapely.geometry.polygon.Polygon at 0x7feec6c13f10>,
  'new_spp': {}},
 14483: {'shape': <shapely.ge

In [120]:
myar.new_spp_table('test_kba_log.csv')

---

In [ ]:
tilas = myfile.getTiles(5, offsetLat=2.5, offsetLon=2.5)

In [ ]:
myfile.index_reg

In [ ]:
for ti in tilas:
    print(myfile.tile2str(ti))

In [ ]:
mysols = pydata.metasearchAlt(tilas, 0.2, 100, 20, 1)

for so in mysols:
    print(so.score, "-", so.ndmScore, "-", so.aggrScore)
    print("Islands: ", pydata.islNum(so))
    if not pydata.isCont(so):
        print("NOT CONTINUOUS!!!")
    print(so.toBitList())
    print(myfile.tile2str(so))
    

In [ ]:
mysols[0].originY

In [ ]:
mysols[0].originY = -90

In [ ]:
tarbits = [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
           [0, 0, 0, 1, 1, 1, 0, 0, 0, 0]]
counts = {0:0, 1:0, 2:0, 3:0}
#ndmabs = np.linspace(0, 50, 20).tolist()
#ndmouts = np.linspace(0, 1, 20).tolist()
ndmwei = np.linspace(10, 30, 20).tolist()
bffr = "index,weigth,ar0,ar1,ar2,ar3\n"
counter = 0

for w in ndmwei:            
    mysols = pyData.metasearchAlt(tilas, 0.2, 100, 20, w)
    tline = "{0},{1},".format(counter, w)
    for ita , ta in enumerate(tarbits):
        bit = "nan,"
        for iso, so in enumerate(mysols):
            if so.toBitList() == ta:
                counts[ita] += 1
                bit = "{0},".format(iso)
                break
        tline += bit
    tline = tline[:-1]
    bffr += tline + "\n"
    counter += 1
            
#with open('exp0.csv', 'w') as fhandle:
#with open('exp1.csv', 'w') as fhandle:
#    fhandle.write(bffr)

In [ ]:
print(bffr)

In [ ]:
exp0 = pd.read_csv('exp0.csv',index_col=0)

In [ ]:
exp0[['weigth', 'ar0', 'ar1', 'ar2', 'ar3']].groupby('weigth').mean()

In [ ]:
counts

/* keys are indexes in observations vector. Values are vectors integers that 
	associates to KBA criteria. Code options are:  
		0 = A1a  
		1 = A1b  
		2 = A1c  
		3 = A1d  
		4 = A1e  
		5 = B1  
		6 = B2*/ 

In [ ]:
myfile.rows, myfile.cols

In [ ]:
for r in range(myfile.rows):
    for c in range(myfile.cols):
        print("{0:.1f}|".format(myfile.presence_grid[r][c]), end='')
    print('')